# Исследование объявлений о продаже квартир

В вашем распоряжении данные сервиса Яндекс Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет. Вам нужно научиться определять рыночную стоимость объектов недвижимости. Для этого проведите исследовательский анализ данных и установите параметры, влияющие на цену объектов. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность.

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта и других объектов — эти данные автоматически получены из геосервисов. Количество парков и водоёмов также заполняется без участия пользователя.

### Откройте файл с данными и изучите общую информацию

In [ ]:
import pandas as pd


data = pd.read_csv('./datasets/real_estate_data.csv', sep='\t')

# Прочитаем первые 10 строк таблицы
with pd.option_context('display.max_columns', None): 
    display(data.head(10))

In [ ]:
# Выведем общую информацию о таблице
data.info()

In [ ]:
# Построим гистограммы для каждого столбца
data.hist(figsize=(20, 20), bins=100)

### Выполните предобработку данных

In [ ]:
# Приведём названия столбцов к стандарту snake_case и переименуем столбцы, чтобы они были более понятны
data.columns = ['total_images', 'last_price', 'total_area', 'first_day_exposition', 'rooms', 'ceiling_height', 'floors_total', 'living_area', 'floor', 'is_apartment', 'is_studio', 'is_open_plan', 'kitchen_area', 'balcony_total', 'locality_name', 'airports_nearest_meters', 'city_centers_nearest_meters', 'parks_around_3k_meters', 'parks_nearest_meters', 'ponds_around_3k_meters', 'ponds_nearest_meters', 'days_exposition']

In [ ]:
# Проверим на наличие явных дубликатов
data.duplicated().sum()

In [ ]:
# Определим количество пропусков в каждом столбце
data.isnull().sum()

In [ ]:
# Посмотрим на долю пропусков в каждом столбце
data.isnull().sum() / data.shape[0]

In [ ]:
# Заполним пропуски в столбце 'balcony' нулями, так как скорее всего пропуски означают, что в квартире нет балконов
data['balcony_total'] = data['balcony_total'].fillna(0)

In [ ]:
# Заполним пропуски в столбце 'floors_total' единицами, так как скорее всего пропуски означают, что в доме всего один этаж
data['floors_total'] = data['floors_total'].fillna(1)

In [ ]:
# Заполним пропуски в столбце 'locality_name' строкой 'unknown', так как название населённого пункта неизвестно
data['locality_name'] = data['locality_name'].fillna('unknown')

In [ ]:
# Заполним пропуски в столбце 'parks_around_3k_meters' нулями, так как скорее всего пропуски означают, что в радиусе 3 км нет парков
data['parks_around_3k_meters'] = data['parks_around_3k_meters'].fillna(0)

In [ ]:
# Заполним пропуски в столбце 'ponds_around_3k_meters' нулями, так как скорее всего пропуски означают, что в радиусе 3 км нет водоёмов
data['ponds_around_3k_meters'] = data['ponds_around_3k_meters'].fillna(0)

In [ ]:
# Заполним пропуски в столбце 'days_exposition' нулями, так как скорее всего пропуски означают, что объявление ещё не снято
data['days_exposition'] = data['days_exposition'].fillna(0)

In [ ]:
# Посмотрим на описание столбца 'ceiling_height'
data['ceiling_height'].describe()

In [ ]:
# Видим, что в столбце 'ceiling_height' есть аномально большие значения
# Посмотрим какие уникальные значения есть в столбце 'ceiling_height' и отсортируем их по возрастанию
sorted(data['ceiling_height'].unique())

In [ ]:
# Вряд ли в квартирах может быть высота потолков больше 10 метров
# Разделим значения в столбце 'ceiling_height', которые больше 10 метров на 10
data.loc[data['ceiling_height'] > 10, 'ceiling_height'] = data.loc[data['ceiling_height'] > 10, 'ceiling_height'] / 10

In [ ]:
# Заполним пропуски в столбце 'ceiling_height' медианой, в среднем высота потолков в квартирах не сильно отличается
data['ceiling_height'] = data['ceiling_height'].fillna(data['ceiling_height'].median())

In [ ]:
# Посмотрим на описание столбца 'city_centers_nearest_meters'
data['city_centers_nearest_meters'].describe()

In [ ]:
# Удалим выбросы в столбце 'city_centers_nearest_meters', оставив только значения меньше 30 км и посмотрим на описание столбца 'city_centers_nearest_meters' без выбросов
data.query('city_centers_nearest_meters < 30000')['city_centers_nearest_meters'].describe()

In [ ]:
# Из описание столбца видно, что выбросы в данном случае не сильно меняют значение для медианы, поэтому я не буду их удалять и заполню пропуски медианой
data['city_centers_nearest_meters'] = data['city_centers_nearest_meters'].fillna(data['city_centers_nearest_meters'].median())

In [ ]:
# Посмотрим на описание столбца 'living_area'
data['living_area'].describe()

In [ ]:
# Удалим выбросы в столбце 'living_area', оставив только значения меньше 200 кв. метров и посмотрим на описание столбца 'living_area' без выбросов
data.query('living_area < 200')['living_area'].describe()

In [ ]:
# Из описание столбца видно, что выбросы в данном случае не сильно меняют значение для медианы, поэтому я не буду их удалять и заполню пропуски медианой
data['living_area'] = data['living_area'].fillna(data['living_area'].median())

### Добавьте в таблицу новые столбцы

### Проведите исследовательский анализ данных

### Напишите общий вывод

**Чек-лист готовности проекта**

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Файл с данными открыт.
- [x]  Файл с данными изучен: выведены первые строки, использован метод `info()`, построены гистограммы.
- [x]  Найдены пропущенные значения.
- [ ]  Пропущенные значения заполнены там, где это возможно.
- [ ]  Объяснено, какие пропущенные значения обнаружены.
- [ ]  В каждом столбце установлен корректный тип данных.
- [ ]  Объяснено, в каких столбцах изменён тип данных и почему.
- [ ]  Устранены неявные дубликаты в названиях населённых пунктов.
- [ ]  Обработаны редкие и выбивающиеся значения (аномалии).
- [ ]  В таблицу добавлены новые параметры:
       – цена одного квадратного метра;
       – день публикации объявления (0 - понедельник, 1 - вторник и т. д.);
       – месяц публикации объявления;
       – год публикации объявления;
       – тип этажа квартиры (значения — «первый», «последний», «другой»);
       – расстояние до центра города в километрах.
- [ ]  Изучены и описаны параметры:
        - общая площадь;
        - жилая площадь;
        - площадь кухни;
        - цена объекта;
        - количество комнат;
        - высота потолков;
        - тип этажа квартиры («первый», «последний», «другой»);
        - общее количество этажей в доме;
        - расстояние до центра города в метрах;
        - расстояние до ближайшего парка.
- [ ]  Выполнено задание «Изучите, как быстро продавались квартиры (столбец `days_exposition`)»:
    - построена гистограмма;
    - рассчитаны среднее и медиана;
    - описано, сколько обычно занимает продажа и указано, какие продажи можно считать быстрыми, а какие — необычно долгими.
- [ ]  Выполнено задание «Определите факторы, которые больше всего влияют на общую (полную) стоимость объекта». Построены графики, которые показывают зависимость цены от параметров:
        - общая площадь;
        - жилая площадь;
        - площадь кухни;
        - количество комнат;
        - тип этажа, на котором расположена квартира (первый, последний, другой);
        - дата размещения (день недели, месяц, год).
- [ ]  Выполнено задание «Посчитайте среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений»:
    - выделены населённые пункты с самой высокой и низкой стоимостью квадратного метра.
- [ ]  Выполнено задание «Выделите квартиры в Санкт-Петербурге с помощью столбца `locality_name` и вычислите их среднюю стоимость на разном удалении от центра»:
    -  учтён каждый километр расстояния, известны средние цены квартир в одном километре от центра, в двух и так далее;
    -  описано, как стоимость объекта зависит от расстояния до центра города;
    -  построен график изменения средней цены для каждого километра от центра Петербурга.
- [ ]  На каждом этапе сделаны промежуточные выводы.
- [ ]  В конце проекта сделан общий вывод.

airports_nearest — расстояние до ближайшего аэропорта в метрах (м)
balcony — число балконов
ceiling_height — высота потолков (м)
cityCenters_nearest — расстояние до центра города (м)
days_exposition — сколько дней было размещено объявление (от публикации до снятия)
first_day_exposition — дата публикации
floor — этаж
floors_total — всего этажей в доме
is_apartment — апартаменты (булев тип)
kitchen_area — площадь кухни в квадратных метрах (м²)
last_price — цена на момент снятия с публикации
living_area — жилая площадь в квадратных метрах (м²)
locality_name — название населённого пункта
open_plan — свободная планировка (булев тип)
parks_around3000 — число парков в радиусе 3 км
parks_nearest — расстояние до ближайшего парка (м)
ponds_around3000 — число водоёмов в радиусе 3 км
ponds_nearest — расстояние до ближайшего водоёма (м)
rooms — число комнат
studio — квартира-студия (булев тип)
total_area — общая площадь квартиры в квадратных метрах (м²)
total_images — число фотографий квартиры в объявлении
